<a href="https://colab.research.google.com/github/aivscovid19/data_pipeline/blob/gulnoza/JobDispatcher.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Set up and update**

In [ ]:
from google.colab import auth
#credentials = auth.authenticate_user()

!apt-get update && apt-get upgrade
!apt install chromium-chromedriver
!pip install centaurMiner==0.0.8

!apt autoremove

In [2]:
from datetime import datetime
import pandas as pd
from pandas.io import gbq

class JobDispatcher():
  def __init__(self, credentials, project_id, url_table):
    self.credentials = credentials
    self.project_id = project_id
    self.url_table = url_table
    self.url_schema = [
        {'name': 'article_url', 'type': 'STRING',   'mode': 'REQUIRED'},
        {'name': 'catalog_url', 'type': 'STRING',   'mode': 'REQUIRED'},
        {'name': 'is_pdf',      'type': 'INTEGER',  'mode': 'REQUIRED'},
        {'name': 'language',    'type': 'STRING',                     },
        {'name': 'status',      'type': 'STRING',   'mode': 'REQUIRED'},
        {'name': 'timestamp',   'type': 'DATETIME', 'mode': 'REQUIRED'},
        {'name': 'worker_id',   'type': 'STRING',                     },
        {'name': 'meta_info',   'type': 'STRING',                     },
    ]

  def register_job(self, worker_id, limit):
    """

    Args:
      limit (int): A limit of urls to retrieve from BQ
    """

    query = f"""
      SELECT article_url, catalog_url, is_pdf, language, meta_info 
      FROM (SELECT *, ROW_NUMBER() OVER
            (PARTITION BY article_url
            ORDER BY timestamp DESC) AS rank
            FROM {self.url_table})
      WHERE rank = 1 AND status = "not mined" AND is_pdf = 0
      ORDER BY timestamp
      LIMIT {limit}
    """
    urls_df = pd.read_gbq(query=query, project_id=self.project_id,
                      credentials=self.credentials)
    
    # Update status on URLBuilder table and upload it to BQ
    urls_df['status'] = 'working on'
    urls_df['timestamp'] = datetime.utcnow()
    urls_df['worker_id'] = worker_id
    urls_df.to_gbq(destination_table=self.url_table,
              project_id=self.project_id,
              if_exists='append',
              table_schema=self.url_schema,
              credentials=self.credentials)
    return urls_df
  
  def update_job_status(self, urls_df):
    urls_df['status'] = 'done'
    urls_df['timestamp'] = datetime.utcnow()
    urls_df.to_gbq(destination_table=self.url_table,
              project_id=self.project_id,
              if_exists='append',
              table_schema=self.url_schema,
              credentials=self.credentials)
    print("Done")
